In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='NDX', timeframe='H1', direction='buy', pruner_type='successive',
        train_start=datetime(2018,2,1), train_end=datetime(2025,4,1),
        test_start=datetime(2023,2,1),  test_end=datetime(2024,4,1),
        label_method='random', search_type='reliability', search_subtype='',
        n_models=1, n_jobs=-1, debug=False,
    ),
    # dict(
    #     symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='successive',
    #     train_start=datetime(2018,2,1), train_end=datetime(2025,4,1),
    #     test_start=datetime(2023,2,1),  test_end=datetime(2024,4,1),
    #     label_method='filter', search_type='clusters', search_subtype='kmeans',
    #     n_models=1, n_jobs=8, debug=False,
    # ),
    # dict(
    #     symbol='GDAXI', timeframe='H1', direction='buy', pruner_type='successive',
    #     train_start=datetime(2018,2,1), train_end=datetime(2025,4,1),
    #     test_start=datetime(2023,2,1),  test_end=datetime(2024,4,1),
    #     label_method='filter', search_type='clusters', search_subtype='kmeans',
    #     n_models=1, n_jobs=8, debug=False,
    # ),
]

# Crear tag para cada configuración
for cfg in configs:
    # Construir el tag asegurando que no haya dobles guiones bajos por campos vacíos
    tag_parts = [
        cfg['symbol'],
        cfg['timeframe'],
        cfg['direction'],
        cfg['label_method'][:2],
        cfg['search_type'][:3],
        (cfg.get('search_subtype') or '')[:2],
    ]
    # Filtrar partes vacías y unir con "_"
    cfg["tag"] = "_".join([part for part in tag_parts if part]).strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 5000
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[NDX_H1_buy_ra_rel] modelo 0 trial 1/5000 score=-1.000000 avg=131.26s mem=549.47MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 2/5000 score=-1.000000 avg=139.54s mem=605.88MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 3/5000 score=-1.000000 avg=182.95s mem=644.60MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 4/5000 score=-1.000000 avg=175.11s mem=653.67MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 5/5000 score=-1.000000 avg=155.32s mem=672.97MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 6/5000 score=-1.000000 avg=138.71s mem=666.47MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 7/5000 score=-1.000000 avg=137.43s mem=688.10MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 8/5000 score=-1.000000 avg=145.29s mem=705.34MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 9/5000 score=-1.000000 avg=155.04s mem=706.87MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 10/5000 score=0.426434 avg=156.91s mem=714.75MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 11/5000 score=0.426434 avg=164.50s mem=766.87MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 12/5000 score=0.426434 avg=157.82s mem=76